In [5]:
# %% ================================================================
# 📦 IMPORTACIONES OPTIMIZADAS — SENASOFT MODELO HÍBRIDO (2025)
# ================================================================

# 🔢 Manipulación de datos
import pandas as pd
import numpy as np
from collections import Counter
df = pd.read_csv("dataset_comunidades_senasoft.csv")

# ⚙️ Machine Learning / Modelos
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, precision_recall_curve, classification_report
)
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

# ⚖️ Balanceo de clases
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.combine import SMOTEENN

# 🧠 Deep Learning (TensorFlow / Keras)
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import Input, regularizers

# 📊 Visualización
import plotly.graph_objects as go

# 🚫 Suprimir warnings molestos (opcional)
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)

print("✅ Importaciones completadas correctamente.")
print(f"TensorFlow versión: {tf.__version__}")


✅ Importaciones completadas correctamente.
TensorFlow versión: 2.20.0


In [6]:
# %% ================================================================
# 🧹 LIMPIEZA Y RECLASIFICACIÓN AUTOMÁTICA DE COMENTARIOS + URGENCIA
# ================================================================

import pandas as pd
import numpy as np
import re

# ================================================================
# 1️⃣ Cargar dataset
# ================================================================
df = pd.read_csv("dataset_comunidades_senasoft.csv")

print("="*70)
print("🧹 LIMPIEZA + RECLASIFICACIÓN DE CATEGORÍAS (por contenido de comentario)")
print("="*70)
print(f"📊 Registros iniciales: {len(df)}")

# ================================================================
# 2️⃣ Limpieza básica de espacios y valores nulos comunes
# ================================================================
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].apply(lambda x: x.strip() if isinstance(x, str) else x)

df = df.replace(['', ' ', 'nan', 'NaN', 'None', 'null', 'NULL'], np.nan)

# ================================================================
# 3️⃣ Imputación de valores perdidos (numéricos → mediana, categóricos → moda)
# ================================================================
for col in df.columns:
    if df[col].dtype in [np.float64, np.int64]:
        df[col].fillna(df[col].median(), inplace=True)
    else:
        moda = df[col].mode(dropna=True)
        if not moda.empty:
            df[col].fillna(moda[0], inplace=True)

# ================================================================
# 4️⃣ Normalización de texto (comentarios, categoría, ciudad)
# ================================================================
if "Comentario" in df.columns:
    df["Comentario"] = df["Comentario"].astype(str).str.strip().str.lower()

if "Categoría del problema" in df.columns:
    df["Categoría del problema"] = df["Categoría del problema"].astype(str).str.strip().str.title()

if "Ciudad" in df.columns:
    df["Ciudad"] = df["Ciudad"].astype(str).str.strip().str.title()
    df["Ciudad"].replace('', np.nan, inplace=True)

# ================================================================
# 5️⃣ Diccionario de palabras clave por categoría
# ================================================================
mapa_categorias = {
    "Medio Ambiente": [
        "basura", "contaminación", "río", "ríos", "agua", "árbol", "medio ambiente",
        "residuos", "limpieza", "aire", "plástico", "verde", "naturaleza"
    ],
    "Salud": [
        "salud", "médico", "hospital", "doctor", "clínica", "enfermo", "vacuna", "medicina"
    ],
    "Educación": [
        "escuela", "educación", "colegio", "profesor", "estudiante", "biblioteca", "cultural", "universidad"
    ],
    "Seguridad": [
        "policía", "robo", "delincuencia", "seguridad", "peligroso", "violencia", "ladrones", "presencia policial"
    ]
}

# ================================================================
# 6️⃣ Función para reclasificar según palabras clave del comentario
# ================================================================
def detectar_categoria(comentario):
    if not isinstance(comentario, str) or comentario.strip() == "":
        return np.nan
    comentario = comentario.lower()
    for categoria, palabras in mapa_categorias.items():
        if any(re.search(rf"\b{palabra}\b", comentario) for palabra in palabras):
            return categoria
    return np.nan  # Si no se encuentra ninguna coincidencia

# ================================================================
# 7️⃣ Aplicar detección de categoría
# ================================================================
df["Categoría detectada"] = df["Comentario"].apply(detectar_categoria)

# ================================================================
# 8️⃣ Priorizar la categoría detectada sobre la original
# ================================================================
df["Categoría final"] = df["Categoría detectada"].combine_first(df["Categoría del problema"])

# ================================================================
# 9️⃣ Filtrar categorías válidas
# ================================================================
df = df[df["Categoría final"].isin(["Salud", "Medio Ambiente", "Seguridad", "Educación"])].copy()

# ================================================================
# 🔟 Generación automática del nivel de urgencia (Urgencia_num)
# ================================================================
def detectar_urgencia(texto):
    """
    Asigna 1 (urgente) si el comentario expresa problema o necesidad grave,
    0 (no urgente) en caso contrario.
    """
    if not isinstance(texto, str) or texto.strip() == "":
        return 0

    texto = texto.lower()

    palabras_urgentes = [
        "no hay", "falta", "problema", "urgente", "grave", "peligro", "riesgo", "crítico",
        "contaminación", "basura", "enfermo", "no tenemos", "emergencia"
    ]

    if any(palabra in texto for palabra in palabras_urgentes):
        return 1
    return 0

df["Urgencia_num"] = df["Comentario"].apply(detectar_urgencia)

# ================================================================
# 11️⃣ Filtrar los registros de Medio Ambiente
# ================================================================
ambiente = df[df["Categoría final"] == "Medio Ambiente"].copy().reset_index(drop=True)

# ================================================================
# 12️⃣ Reporte final
# ================================================================
print("\n✅ LIMPIEZA + RECLASIFICACIÓN COMPLETA")
print(f"📊 Registros finales totales: {len(df)}")
print(f"🏷️ Distribución de categorías finales:\n{df['Categoría final'].value_counts()}")

print("\n🧩 Distribución de niveles de urgencia:")
print(df['Urgencia_num'].value_counts(normalize=True).apply(lambda x: f"{x:.2%}"))

print(f"\n🌍 Total registros de Medio Ambiente: {len(ambiente)}")
print(ambiente[["Comentario", "Urgencia_num"]].head(10))


🧹 LIMPIEZA + RECLASIFICACIÓN DE CATEGORÍAS (por contenido de comentario)
📊 Registros iniciales: 10000

✅ LIMPIEZA + RECLASIFICACIÓN COMPLETA
📊 Registros finales totales: 10000
🏷️ Distribución de categorías finales:
Categoría final
Medio Ambiente    4096
Seguridad         2298
Salud             2259
Educación         1347
Name: count, dtype: int64

🧩 Distribución de niveles de urgencia:
Urgencia_num
1    80.93%
0    19.07%
Name: proportion, dtype: object

🌍 Total registros de Medio Ambiente: 4096
                                      Comentario  Urgencia_num
0  no tenemos centros culturales ni bibliotecas.             1
1            las basuras no se recogen a tiempo.             1
2      la contaminación del río está aumentando.             1
3    hay problemas con la recolección de basura.             1
4      la contaminación del río está aumentando.             1
5  necesitamos más acceso a internet en la zona.             0
6            falta agua potable en varias casas.          

In [7]:
ambiente.to_csv("dataset_ambiente_senasoft.csv", index=False)
print("✅ Dataset de ambiente guardado como 'dataset_ambiente_senasoft.csv'")

✅ Dataset de ambiente guardado como 'dataset_ambiente_senasoft.csv'


In [8]:
df['Comentario'].unique()

array(['las calles están muy oscuras y peligrosas.',
       'no tenemos centros culturales ni bibliotecas.',
       'las basuras no se recogen a tiempo.',
       'la contaminación del río está aumentando.',
       'no hay suficientes escuelas públicas.',
       'hay problemas con la recolección de basura.',
       'necesitamos más acceso a internet en la zona.',
       'faltan médicos en el centro de salud.',
       'falta agua potable en varias casas.',
       'queremos más presencia policial.'], dtype=object)

In [9]:
df['Ciudad'].unique()

array(['Manizales', 'Santa Marta', 'Medellín', 'Bogotá', 'Cartagena',
       'Cali', 'Barranquilla', 'Pereira', 'Cúcuta', 'Bucaramanga'],
      dtype=object)

In [10]:
df.isnull().mean() * 100

ID                               0.00
Nombre                           0.00
Edad                             0.00
Género                           0.00
Ciudad                           0.00
Comentario                       0.00
Categoría del problema           0.00
Nivel de urgencia                0.00
Fecha del reporte                0.00
Acceso a internet                0.00
Atención previa del gobierno     0.00
Zona rural                       0.00
Categoría detectada             53.41
Categoría final                  0.00
Urgencia_num                     0.00
dtype: float64

In [11]:
# %%
def crear_features_efectivas(df):
    df = df.copy()
    df["Zona rural"] = df["Zona rural"].astype(int)
    df["Acceso a internet"] = df["Acceso a internet"].astype(int)
    df["Atención previa del gobierno"] = df["Atención previa del gobierno"].astype(int)
    df["Edad"] = df["Edad"].astype(float)

    df["Vulnerabilidad_Total"] = (
        df["Zona rural"] * 3 +
        (1 - df["Acceso a internet"]) * 2 +
        (1 - df["Atención previa del gobierno"]) * 2.5
    )
    df["Edad_Normalizada"] = df["Edad"] / 100
    df["Es_Vulnerable_Edad"] = ((df["Edad"] < 18) | (df["Edad"] > 65)).astype(int)
    df["Rural_Sin_Internet"] = ((df["Zona rural"] == 1) & (df["Acceso a internet"] == 0)).astype(int)
    df["Desatendido"] = (df["Atención previa del gobierno"] == 0).astype(int)
    df["Desatendido_Rural"] = df["Desatendido"] * df["Zona rural"]
    df["Edad_Rural"] = df["Edad"] * df["Zona rural"]
    df["Internet_Atencion"] = df["Acceso a internet"] * df["Atención previa del gobierno"]
    return df


In [12]:
def build_keras_model_regularizado(input_dim):
    """
    🧠 Modelo Keras optimizado para ALTA PRECISIÓN:
    - Regularización L2 y Dropout controlado
    - Focal Loss (mejor manejo de clases minoritarias)
    - Métricas: precisión, recall, accuracy
    - Optimizador AdamW con decaimiento de learning rate
    """
    from tensorflow.keras import backend as K

    # 🔹 Definir Focal Loss personalizada
    def focal_loss(gamma=2., alpha=.25):
        def focal_loss_fixed(y_true, y_pred):
            y_true = K.cast(y_true, K.floatx())
            bce = K.binary_crossentropy(y_true, y_pred)
            bce_exp = K.exp(-bce)
            focal_loss = alpha * K.pow((1 - bce_exp), gamma) * bce
            return K.mean(focal_loss)
        return focal_loss_fixed

    # 🔹 Arquitectura
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.005)),
        BatchNormalization(),
        Dropout(0.25),

        Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.005)),
        BatchNormalization(),
        Dropout(0.2),

        Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.005)),
        BatchNormalization(),

        Dense(1, activation='sigmoid')
    ])

    # 🔹 Optimizador AdamW (más estable)
    opt = tf.keras.optimizers.AdamW(learning_rate=0.001, weight_decay=1e-5)

    # 🔹 Compilación con Focal Loss
    model.compile(
        optimizer=opt,
        loss=focal_loss(gamma=2.0, alpha=0.25),
        metrics=[
            tf.keras.metrics.Precision(name="precision"),
            tf.keras.metrics.Recall(name="recall"),
            tf.keras.metrics.BinaryAccuracy(name="accuracy")
        ]
    )

    return model


In [13]:
def entrenar_hibrido_mejor_precision(ciudad, metodo_balanceo='smoteenn', random_state=42):
    """
    Entrenamiento híbrido optimizado para mejorar PRECISIÓN:
    - Regularización fuerte + Dropout
    - Focal Loss
    - Balanceo SMOTE o SMOTEENN
    - Umbral adaptativo con precisión mínima deseada (default: 0.9)
    """

    df_ciudad = ambiente[ambiente["Ciudad"].str.lower() == ciudad.lower()].copy()
    if df_ciudad.empty:
        print(f"\n❌ No hay registros para {ciudad}")
        return None

    print(f"\n{'='*70}")
    print(f"📍 ENTRENANDO MODELO PARA CIUDAD: {ciudad.upper()}")
    print(f"{'='*70}")

    df_ciudad = crear_features_efectivas(df_ciudad)

    X = df_ciudad.select_dtypes(include=[np.number]).drop(columns=["Urgencia_num"], errors="ignore")
    y = df_ciudad["Urgencia_num"].astype(int)

    X = X.replace([np.inf, -np.inf], np.nan).dropna()
    y = y.loc[X.index]

    if len(y) < 10 or len(np.unique(y)) < 2:
        print("⚠️ Datos insuficientes para entrenamiento en esta ciudad.")
        return None

    # 🧩 Balanceo
    min_class = y.value_counts().min()
    if metodo_balanceo.lower() == "smote" and min_class >= 6:
        sm = SMOTE(random_state=random_state)
        X, y = sm.fit_resample(X, y)
        print(f"✅ Balanceo con SMOTE → Clase 0: {sum(y==0)}, Clase 1: {sum(y==1)}")
    elif metodo_balanceo.lower() == "smoteenn" and min_class >= 6:
        smenn = SMOTEENN(random_state=random_state)
        X, y = smenn.fit_resample(X, y)
        print(f"✅ Balanceo con SMOTEENN → Clase 0: {sum(y==0)}, Clase 1: {sum(y==1)}")
    else:
        print("⚠️ No se aplicó balanceo (datos insuficientes o método distinto).")

    # 🧮 Escalado
    scaler = RobustScaler()
    X_scaled = scaler.fit_transform(X)

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
    f1_scores, prec_scores, rec_scores, acc_scores = [], [], [], []
    thresholds = []

    for fold, (train_idx, test_idx) in enumerate(skf.split(X_scaled, y), 1):
        print(f"\n📂 Fold {fold}/5")

        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
        class_weights = dict(zip(np.unique(y_train), weights))

        model = build_keras_model_regularizado(input_dim=X_train.shape[1])

        callbacks = [
            tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=8, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3)
        ]

        model.fit(
            X_train, y_train,
            validation_data=(X_test, y_test),
            epochs=80,
            batch_size=16,
            verbose=0,
            callbacks=callbacks,
            class_weight=class_weights
        )

        y_pred_proba = model.predict(X_test).ravel()
        prec_arr, rec_arr, th_arr = precision_recall_curve(y_test, y_pred_proba)

        # 🎯 Priorizar precisión
        desired_prec = 0.9
        valid_idx = np.where(prec_arr >= desired_prec)[0]
        if len(valid_idx) > 0:
            best_idx = valid_idx[np.argmax(rec_arr[valid_idx])]
        else:
            best_idx = np.argmax(2 * prec_arr * rec_arr / (prec_arr + rec_arr + 1e-8))

        best_th = th_arr[best_idx] if best_idx < len(th_arr) else 0.5
        thresholds.append(best_th)

        y_pred_opt = (y_pred_proba >= best_th).astype(int)
        f1 = f1_score(y_test, y_pred_opt)
        acc = accuracy_score(y_test, y_pred_opt)
        prec_val = precision_score(y_test, y_pred_opt)
        rec_val = recall_score(y_test, y_pred_opt)

        f1_scores.append(f1)
        acc_scores.append(acc)
        prec_scores.append(prec_val)
        rec_scores.append(rec_val)

        print(f"🔹 Fold {fold} → F1: {f1:.3f} | Acc: {acc:.3f} | Prec: {prec_val:.3f} | Rec: {rec_val:.3f} | Th: {best_th:.2f}")

    avg_threshold = np.mean(thresholds)
    print(f"\n🔧 Umbral promedio (prec ≥ 0.9): {avg_threshold:.3f}")

    # 🧩 Entrenamiento final
    model_final = build_keras_model_regularizado(input_dim=X_scaled.shape[1])
    model_final.fit(X_scaled, y, epochs=80, batch_size=16, verbose=0)

    y_pred_proba_full = model_final.predict(X_scaled).ravel()
    y_pred_final = (y_pred_proba_full >= avg_threshold).astype(int)

    print(f"\n🔧 Umbral global: {avg_threshold:.3f}")
    print("📊 Reporte final:")
    print(classification_report(y, y_pred_final, target_names=["No Urgente", "Urgente"]))
    print("📉 Matriz de confusión:")
    print(confusion_matrix(y, y_pred_final))

    print(f"\n📈 F1 promedio: {np.mean(f1_scores):.3f}")
    print(f"🎯 Precisión promedio: {np.mean(prec_scores):.3f}")
    print(f"📉 Recall promedio: {np.mean(rec_scores):.3f}")
    print(f"✅ Exactitud promedio: {np.mean(acc_scores):.3f}")

    print(f"\n🏁 Entrenamiento completado para {ciudad.upper()} ✅")
    return model_final, scaler, avg_threshold

In [14]:
# %%
print("\n" + "="*70)
print("🚀 ENTRENAMIENTO HÍBRIDO (scikit-learn + Keras)")
print("="*70)

ciudades = ["Pereira", "Bogotá", "Cali"]
resultados = {}

for ciudad in ciudades:
    print(f"\n🌆 Ciudad: {ciudad}")
    try:
        modelo, escalador, umbral = entrenar_hibrido_mejor_precision(ciudad, metodo_balanceo='smoteenn')

        # Guardar resultados por ciudad
        resultados[ciudad] = {
            "modelo": modelo,
            "escalador": escalador,
            "umbral": umbral
        }

        print(f"✅ {ciudad} entrenada correctamente. Umbral óptimo: {umbral:.3f}")
    except Exception as e:
        print(f"❌ Error entrenando {ciudad}: {str(e)}")

    print(f"\n{'-'*70}\n")

print("🏁 ENTRENAMIENTO COMPLETADO PARA TODAS LAS CIUDADES")



🚀 ENTRENAMIENTO HÍBRIDO (scikit-learn + Keras)

🌆 Ciudad: Pereira

📍 ENTRENANDO MODELO PARA CIUDAD: PEREIRA
✅ Balanceo con SMOTEENN → Clase 0: 128, Clase 1: 82

📂 Fold 1/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 181ms/step
🔹 Fold 1 → F1: 0.867 | Acc: 0.905 | Prec: 0.929 | Rec: 0.812 | Th: 0.56

📂 Fold 2/5
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
🔹 Fold 2 → F1: 0.741 | Acc: 0.833 | Prec: 0.909 | Rec: 0.625 | Th: 0.60

📂 Fold 3/5
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 178ms/stepWARNING:tensorflow:6 out of the last 6 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x000002493448C860> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unneces

In [15]:
# ====================================================
# 📊 Visualización por Ciudad (con Plotly y widgets)
# ====================================================
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact, widgets, interactive_output
from IPython.display import display, clear_output

# --- Cargar dataset base ---
df = pd.read_csv("dataset_comunidades_senasoft.csv")

# --- Filtrar solo Medio Ambiente ---
ambiente_viz = df[df["Categoría del problema"].str.lower() == "medio ambiente"].copy()

# --- Limpieza básica ---
ambiente_viz = ambiente_viz.dropna(subset=["Ciudad", "Edad", "Nivel de urgencia"])
ambiente_viz["Edad"] = pd.to_numeric(ambiente_viz["Edad"], errors="coerce")
ambiente_viz = ambiente_viz.dropna(subset=["Edad"])
ambiente_viz["Nivel de urgencia"] = ambiente_viz["Nivel de urgencia"].str.lower().str.strip()

# --- Crear feature de vulnerabilidad (simplificada) ---
def crear_features_viz(df):
    df = df.copy()
    df["Acceso a internet"] = pd.to_numeric(df["Acceso a internet"], errors="coerce").fillna(0).astype(int)
    df["Atención previa del gobierno"] = pd.to_numeric(df["Atención previa del gobierno"], errors="coerce").fillna(0).astype(int)
    df["Zona rural"] = pd.to_numeric(df["Zona rural"], errors="coerce").fillna(0).astype(int)
    df["Vulnerabilidad_Total"] = (
        df["Zona rural"] * 3 +
        (1 - df["Acceso a internet"]) * 2 +
        (1 - df["Atención previa del gobierno"]) * 2.5
    )
    return df

ambiente_viz = crear_features_viz(ambiente_viz)

# --- Coordenadas aproximadas de ciudades ---
coords_ciudades = {
    "Bogotá": (4.7110, -74.0721),
    "Medellín": (6.2442, -75.5812),
    "Cali": (3.4516, -76.5320),
    "Barranquilla": (10.9639, -74.7964),
    "Cartagena": (10.3910, -75.4794),
    "Bucaramanga": (7.1193, -73.1227),
    "Pereira": (4.8143, -75.6946),
    "Manizales": (5.0703, -75.5138),
    "Cúcuta": (7.8939, -72.5078),
    "Ibagué": (4.4389, -75.2322),
    "Neiva": (2.9386, -75.2819),
    "Villavicencio": (4.1420, -73.6266),
    "Santa Marta": (11.2408, -74.1990),
    "Popayán": (2.4448, -76.6147),
    "Montería": (8.7489, -75.8814),
    "Pasto": (1.2136, -77.2811)
}
ambiente_viz["lat"] = ambiente_viz["Ciudad"].map(lambda c: coords_ciudades.get(c, (None, None))[0])
ambiente_viz["lon"] = ambiente_viz["Ciudad"].map(lambda c: coords_ciudades.get(c, (None, None))[1])

# --- Ciudades disponibles ---
ciudades_disponibles = sorted(ambiente_viz["Ciudad"].dropna().unique())

# --- Función principal de visualización ---
def visualizar_ciudad(ciudad):
    clear_output(wait=True)
    print(f"📍 Ciudad seleccionada: {ciudad}")

    dfc = ambiente_viz[ambiente_viz["Ciudad"] == ciudad]
    if dfc.empty:
        print("❌ No hay registros para esta ciudad.")
        return

    # 1️⃣ Distribución de nivel de urgencia
    urg_counts = dfc["Nivel de urgencia"].value_counts().reset_index()
    urg_counts.columns = ["Nivel de urgencia", "Cantidad"]
    fig1 = px.bar(
        urg_counts, x="Nivel de urgencia", y="Cantidad",
        color="Nivel de urgencia", text="Cantidad",
        title=f"Distribución de Nivel de Urgencia - {ciudad}",
        template="plotly_white"
    )
    fig1.update_traces(textposition="outside")
    fig1.show()

    # 2️⃣ Distribución por edad
    fig2 = px.histogram(
        dfc, x="Edad", nbins=20, color="Nivel de urgencia",
        marginal="box",
        title=f"Distribución de Edades por Urgencia - {ciudad}",
        template="plotly_white"
    )
    fig2.show()

    # 7️⃣ Promedio de vulnerabilidad total por ciudad
    vuln_city = (
        ambiente_viz.groupby("Ciudad")["Vulnerabilidad_Total"]
        .mean().reset_index().sort_values(by="Vulnerabilidad_Total", ascending=False)
    )
    fig3 = px.bar(
        vuln_city, x="Vulnerabilidad_Total", y="Ciudad",
        orientation="h", text="Vulnerabilidad_Total",
        title="Promedio de Vulnerabilidad Total por Ciudad",
        template="plotly_white", color="Vulnerabilidad_Total"
    )
    fig3.update_traces(texttemplate="%{text:.2f}", textposition="outside")
    fig3.show()

    # 8️⃣ Relación Edad vs Vulnerabilidad Total
    fig4 = px.scatter(
        dfc, x="Edad", y="Vulnerabilidad_Total", color="Nivel de urgencia",
        symbol="Zona rural",
        hover_data=["Acceso a internet", "Atención previa del gobierno"],
        title=f"Edad vs Vulnerabilidad Total - {ciudad}",
        template="plotly_white"
    )
    fig4.show()

    # 9️⃣ Mapa geográfico de calor (casos por ciudad)
    casos_ciudad = (
        ambiente_viz.groupby(["Ciudad", "lat", "lon"])
        .size().reset_index(name="Casos")
    )
    casos_ciudad = casos_ciudad.dropna(subset=["lat", "lon"])

    fig5 = px.scatter_mapbox(
        casos_ciudad,
        lat="lat", lon="lon", size="Casos",
        color="Casos", hover_name="Ciudad",
        zoom=4.3,
        color_continuous_scale="Viridis",
        mapbox_style="carto-positron",
        title="Mapa geográfico de calor - Casos por ciudad"
    )
    fig5.show()

# --- Selector interactivo manual ---
selector = widgets.Dropdown(
    options=ciudades_disponibles,
    description='Ciudad:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)
display(selector)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        visualizar_ciudad(change['new'])

selector.observe(on_change)
print("✅ Selecciona una ciudad en el menú desplegable para ver las gráficas.")


In [14]:
import joblib

joblib.dump(resultados["Pereira"]["modelo"], "modelo_urgencia_pereira.keras")
joblib.dump(resultados["Pereira"]["escalador"], "scaler_pereira.pkl")
joblib.dump(resultados["Pereira"]["umbral"], "umbral_pereira.pkl")


['umbral_pereira.pkl']

In [16]:
modelo_final, scaler, avg_threshold = entrenar_hibrido_mejor_precision("Bogotá")


📍 ENTRENANDO MODELO PARA CIUDAD: BOGOTÁ
✅ Balanceo con SMOTEENN → Clase 0: 212, Clase 1: 166

📂 Fold 1/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
🔹 Fold 1 → F1: 0.909 | Acc: 0.921 | Prec: 0.909 | Rec: 0.909 | Th: 0.51

📂 Fold 2/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
🔹 Fold 2 → F1: 0.909 | Acc: 0.921 | Prec: 0.909 | Rec: 0.909 | Th: 0.51

📂 Fold 3/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
🔹 Fold 3 → F1: 0.912 | Acc: 0.921 | Prec: 0.912 | Rec: 0.912 | Th: 0.53

📂 Fold 4/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
🔹 Fold 4 → F1: 0.875 | Acc: 0.893 | Prec: 0.903 | Rec: 0.848 | Th: 0.52

📂 Fold 5/5
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
🔹 Fold 5 → F1: 0.419 | Acc: 0.667 | Prec: 0.900 | Rec: 0.273 | Th: 0.75

🔧 Umbral promedio (prec ≥ 0.9): 0.563
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

🔧 Umbral global: 0.563
📊 Reporte final:
              precision    recall  f1-score   support

  No Urgente       0.91      0.94      0.93       212
     Urgente       0.92      0.88      0.90       166

    accuracy

In [17]:
import pickle
import os

os.makedirs("modelos", exist_ok=True)

# Guardar el modelo Keras
modelo_final.save("modelos/modelo_urgencia_bogota.keras")

# Guardar el escalador (scaler)
with open("modelos/escalador_bogota.pkl", "wb") as f:
    pickle.dump(scaler, f)

# Guardar el umbral
with open("modelos/umbral_bogota.pkl", "w") as f:
    f.write(str(avg_threshold))

print("✅ Archivos del modelo de Pereira guardados correctamente.")


✅ Archivos del modelo de Pereira guardados correctamente.
